In [1]:
import json
import os
import os.path
import pandas as pd

In [6]:
# Get image metadata file path
xmllist = []
for dirpath, dirnames, filenames in os.walk(os.path.join('..', 'hc', 'meta')):
    for filename in filenames:
        if dirpath == os.path.join('..', 'hc', 'meta'):
            continue
        xmllist.append(os.path.join(dirpath, filename))

In [9]:
xmllist

['../hc/meta/3057/SAG_3D_T1/2011-02-01_08_53_10.0/I224530/S102093I224530.xml',
 '../hc/meta/3115/MPRAGE_GRAPPA/2012-07-18_14_24_27.0/I337535/S169231I337535.xml',
 '../hc/meta/3115/MPRAGE_GRAPPA/2011-08-01_13_30_13.0/I253995/S120955I253995.xml',
 '../hc/meta/3857/MPRAGE_GRAPPA/2013-11-13_12_13_49.0/I414646/S212792I414646.xml',
 '../hc/meta/3857/MPRAGE_GRAPPA/2011-09-22_08_10_09.0/I395284/S204398I395284.xml',
 '../hc/meta/3857/MPRAGE_GRAPPA/2012-10-19_10_20_16.0/I395287/S204401I395287.xml',
 '../hc/meta/3768/MPRAGE_GRAPPA/2011-12-21_08_36_30.0/I301540/S148984I301540.xml',
 '../hc/meta/3768/MPRAGE_GRAPPA/2013-01-16_07_56_24.0/I357765/S181445I357765.xml',
 '../hc/meta/3188/SAG_MPRAGE_GRAPPA/2013-04-03_15_31_34.0/I367304/S187032I367304.xml',
 '../hc/meta/3188/SAG_MPRAGE_GRAPPA/2014-12-04_11_13_36.0/I468920/S245310I468920.xml',
 '../hc/meta/3765/MPRAGE_GRAPPA/2012-11-14_08_02_55.0/I353654/S179160I353654.xml',
 '../hc/meta/3765/MPRAGE_GRAPPA/2011-11-30_08_18_44.0/I288122/S142703I288122.xml',


In [41]:
# Generate image dataframe by PATNO and IMG_ID
img_path = []
from xml.dom import minidom
for xml in xmllist:
    root = minidom.parse(xml).documentElement
    subject = root.getElementsByTagName('subject')[0].getAttribute('id')
    study = root.getElementsByTagName('study')[0].getAttribute('uid')
    series = root.getElementsByTagName('series')[0].getAttribute('uid')
    image = root.getElementsByTagName('image')[0].getAttribute('uid')
    relative_path = os.path.join(xml.split(os.sep)[3], xml.split(os.sep)[4], xml.split(os.sep)[5], series)
    img_path.append({'PATNO': int(subject), 'IMG_ID': str(image), 'SERIES': str(series), 'IMG_REL_PATH': str(relative_path)})
img_path = pd.DataFrame(img_path)
img_path['IMG_REL_PATH'] = img_path['IMG_REL_PATH'].apply(lambda x: os.path.join('..', 'hc', 'raw', x))

In [35]:
# Extract nii file by path
import shutil
def move_nii(rec):
    orig_path = rec.IMG_REL_PATH
    orig_path = os.path.join(orig_path, os.listdir(rec.IMG_REL_PATH)[0])
    dest_path = os.path.join('..', 'hc', 'img', str(rec.PATNO)+str(rec.IMG_ID))
    os.mkdir(dest_path)
    dest_path = os.path.join(dest_path, 't1.nii')
    shutil.copyfile(orig_path, dest_path)
    return dest_path

In [69]:
img_path.loc[img_path['PATNO'] == 172084, 'IMG_REL_PATH'] = '../hc/raw/172084/3D_T1-weighted/2022-10-19_14_19_48.0/I1670522'
img_path.loc[img_path['PATNO'] == 187759, 'IMG_REL_PATH'] = '../hc/raw/187759/3D_T1-weighted/2023-02-28_09_47_14.0/I1678671'
img_path.loc[img_path['PATNO'] == 157589, 'IMG_REL_PATH'] = '../hc/raw/157589/3D_T1-weighted/2022-07-06_15_38_22.0/I1617121'
img_path.loc[img_path['PATNO'] == 162905, 'IMG_REL_PATH'] = '../hc/raw/162905/3D_T1-weighted/2022-08-11_12_46_15.0/I1679616'
img_path.loc[img_path['PATNO'] == 137499, 'IMG_REL_PATH'] = '../hc/raw/137499/SAG_3D_T1_MPRAGE/2022-07-11_07_39_29.0/I1616745'
img_path.loc[img_path['PATNO'] == 141714, 'IMG_REL_PATH'] = '../hc/raw/141714/3D_T1/2022-05-24_09_15_11.0/I1616839'
img_path.loc[img_path['PATNO'] == 153233, 'IMG_REL_PATH'] = '../hc/raw/153233/SAG_3D_T1-weighted/2022-11-28_07_13_28.0/I1660562'
img_path.loc[img_path['PATNO'] == 149006, 'IMG_REL_PATH'] = '../hc/raw/149006/3D_T1-weighted/2022-07-14_13_51_20.0/I1617083'
img_path.loc[img_path['PATNO'] == 157434, 'IMG_REL_PATH'] = '../hc/raw/157434/MPRAGE_-_Sag/2022-09-13_09_19_56.0/I1621890'
img_path.loc[img_path['PATNO'] == 156459, 'IMG_REL_PATH'] = '../hc/raw/156459/3D_T1-weighted/2022-11-29_08_59_06.0/I1652981'

In [70]:
img_path['NII_PATH'] = img_path.apply(move_nii, axis=1)

In [72]:
img_path['ROOT'] = img_path['NII_PATH'].apply(lambda x: os.path.dirname(x))

In [93]:
img_path = img_path[['PATNO', 'IMG_ID', 'SERIES', 'ROOT', 'NII_PATH']]

In [79]:
data

,PATNO,IMG_ID,SERIES,ROOT,NII_PATH
0,3057,I224530,S102093,../hc/img/3057I224530,../hc/img/3057I224530/t1.nii
1,3115,I337535,S169231,../hc/img/3115I337535,../hc/img/3115I337535/t1.nii
2,3115,I253995,S120955,../hc/img/3115I253995,../hc/img/3115I253995/t1.nii
3,3857,I414646,S212792,../hc/img/3857I414646,../hc/img/3857I414646/t1.nii
4,3857,I395284,S204398,../hc/img/3857I395284,../hc/img/3857I395284/t1.nii
...,...,...,...,...,...
288,3428,I368571,S187825,../hc/img/3428I368571,../hc/img/3428I368571/t1.nii
289,3156,I243615,S113836,../hc/img/3156I243615,../hc/img/3156I243615/t1.nii
290,4010,I340891,S171296,../hc/img/4010I340891,../hc/img/4010I340891/t1.nii
291,4010,I288213,S142730,../hc/img/4010I288213,../hc/img/4010I288213/t1.nii


In [96]:
meta = pd.read_csv(os.path.join('..', 'hc', 'HCT1_3_29_2023.csv'))[['Image Data ID', 'Subject', 'Sex', 'Age']]
meta.rename(columns={'Image Data ID': 'IMG_ID', 'Subject': 'PATNO'}, inplace=True)
data = pd.merge(img_path, meta, on=['PATNO', 'IMG_ID'])

In [98]:
import json
data_json = data.to_dict(orient='records')
with open('hc.json', 'w+') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

In [77]:
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.fsl import BET
from nipype.interfaces.ants import RegistrationSynQuick
import nipype.interfaces.io as nio

src = Node(nio.DataGrabber(infields=['path'], outfields=['in_files']), name='src')
src.inputs.base_directory = os.path.abspath('.')
src.inputs.template = '%s'
src.inputs.sort_filelist = False
src.inputs.path = img_path['NII_PATH'].tolist()

bet = MapNode(BET(), name='BET', iterfield=['in_file'])
bet.inputs.robust = True # -R

reg = MapNode(RegistrationSynQuick(), name='RegistrationSynQuick', iterfield=['moving_image'])
reg.inputs.fixed_image = os.path.abspath('bin/template.nii.gz') # -f
reg.inputs.output_prefix = 'reg' # -o
reg.inputs.num_threads = 16 # -n

wf = Workflow(name='hc', base_dir='tmp')
wf.connect([
    (src, bet, [('in_files', 'in_file')]),
    (bet, reg, [('out_file', 'moving_image')])
])

In [ ]:
wf.run(plugin='MultiProc', plugin_args={'n_procs': 8})